In [ ]:
# Load system libraries.
import os
import re
import collections
import functools

# Libraries for data wrangling.
import numpy as np
import pandas as pd
import scipy as sc
import sklearn
import xgboost as xgb
import joblib
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

# Visualization libraries.
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sns.set_context('poster')
sns.set(rc={'figure.figsize': (9, 3)})
sns.set_style('whitegrid')
tqdm = functools.partial(tqdm.tqdm, file=sys.stdout, position=0, leave=True)

# Local modules.
%load_ext autoreload
%autoreload 2
